<a href="https://colab.research.google.com/github/moe18/Image-Based-Airbnb-Pricing-Algorithm/blob/master/Stock_Market_Predictor_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Market Predictor with LSTM

**Summary:** will use historical stock data and LSTM to try and predict if a stock will go up or down in the next few days.(DO NOT BASE ANY INVESTMENT DECISION UPON THIS ALGORITHM)



## Get the Data
- will get 5 years of data from yahoo finance 
  - Open price
  - Close price
  - High price
  - Low price
  - Volume
  - Adj Close
  
 In this code i will be looking at three index funds SPY, DIA, and QQQ. In order to try and predict if SPY will go up in 3 days

In [0]:
import pandas as pd
import numpy as np

import pandas_datareader.data as web
from datetime import datetime

In [2]:


start = datetime(2010, 3, 1)

end = datetime(2018, 3, 1)

stocks = ['SPY','DIA','QQQ']

f = web.DataReader(stocks, 'yahoo', start, end)

f.head()

Attributes        High                                Low             \
Symbols            DIA        QQQ         SPY         DIA        QQQ   
Date                                                                   
2010-03-01  104.199997  45.490002  112.000000  103.580002  44.950001   
2010-03-02  104.639999  45.849998  112.739998  103.949997  45.450001   
2010-03-03  104.769997  45.820000  112.970001  103.830002  45.430000   
2010-03-04  104.570000  45.810001  112.800003  103.959999  45.400002   
2010-03-05  105.769997  46.540001  114.339996  104.870003  45.990002   

Attributes                    Open                              Close  \
Symbols            SPY         DIA        QQQ         SPY         DIA   
Date                                                                    
2010-03-01  111.169998  103.669998  44.959999  111.199997  104.019997   
2010-03-02  112.000000  104.430000  45.570000  112.370003  104.010002   
2010-03-03  112.019997  104.349998  45.660000  112.489998  104.089996   
2010-03-04  112.029999  104.239998  45.630001  112.449997  104.500000   
2010-03-05  113.099998  104.989998  46.090000  113.370003  105.760002   

Attributes                            Volume                           \
Symbols           QQQ         SPY        DIA         QQQ          SPY   
Date                                                                    
2010-03-01  45.410000  111.889999  5818700.0  77290100.0  147709700.0   
2010-03-02  45.549999  112.199997  5673300.0  76368500.0  160992400.0   
2010-03-03  45.599998  112.300003  5891900.0  65735500.0  150785000.0   
2010-03-04  45.750000  112.639999  6168300.0  54470900.0  135770400.0   
2010-03-05  46.439999  114.250000  6753500.0  68136300.0  176118800.0   

Attributes  Adj Close                        
Symbols           DIA        QQQ        SPY  
Date                                         
2010-03-01  83.919640  41.234989  92.934830  
2010-03-02  83.911613  41.362122  93.192322  
2010-03-03  83.976173  41.407520  93.275368  
2010-03-04  84.306938  41.543732  93.557755  
2010-03-05  85.323486  42.170288  94.895035

Will only look at the close price, volume, high price, and low price

In [0]:
df = f[['Close','Volume','High','Low']] 

## Preparing the Data

In [0]:
SEQ_LEN = 30 # number of periods to look back at
PERIODS_FORWARD = 3 # number of periods in the future the model will look forward to 
STOCK = 'SPY' # the option price for this stock will be predicted

In [5]:
# make a future colume that shows where the stock will be in 3 days
df['future'] = df['Close'][STOCK].shift(-PERIODS_FORWARD)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
df.head()

Attributes       Close                            Volume              \
Symbols            DIA        QQQ         SPY        DIA         QQQ   
Date                                                                   
2010-03-01  104.019997  45.410000  111.889999  5818700.0  77290100.0   
2010-03-02  104.010002  45.549999  112.199997  5673300.0  76368500.0   
2010-03-03  104.089996  45.599998  112.300003  5891900.0  65735500.0   
2010-03-04  104.500000  45.750000  112.639999  6168300.0  54470900.0   
2010-03-05  105.760002  46.439999  114.250000  6753500.0  68136300.0   

Attributes                     High                                Low  \
Symbols             SPY         DIA        QQQ         SPY         DIA   
Date                                                                     
2010-03-01  147709700.0  104.199997  45.490002  112.000000  103.580002   
2010-03-02  160992400.0  104.639999  45.849998  112.739998  103.949997   
2010-03-03  150785000.0  104.769997  45.820000  112.970001  103.830002   
2010-03-04  135770400.0  104.570000  45.810001  112.800003  103.959999   
2010-03-05  176118800.0  105.769997  46.540001  114.339996  104.870003   

Attributes                             future  
Symbols           QQQ         SPY              
Date                                           
2010-03-01  44.950001  111.169998  112.639999  
2010-03-02  45.450001  112.000000  114.250000  
2010-03-03  45.430000  112.019997  114.269997  
2010-03-04  45.400002  112.029999  114.459999  
2010-03-05  45.990002  113.099998  114.970001

In [0]:
# a function to determin if a stock went up in the past three days 
def classify(current, future):
  if float(future) > float(current):
    return 1
  else:
    return 0

In [0]:
# create a target col
df['target'] = list(map(classify,df['Close'][STOCK],df['future']))

In [89]:
df.head()

Attributes       Close                            Volume              \
Symbols            DIA        QQQ         SPY        DIA         QQQ   
Date                                                                   
2010-03-01  104.019997  45.410000  111.889999  5818700.0  77290100.0   
2010-03-02  104.010002  45.549999  112.199997  5673300.0  76368500.0   
2010-03-03  104.089996  45.599998  112.300003  5891900.0  65735500.0   
2010-03-04  104.500000  45.750000  112.639999  6168300.0  54470900.0   
2010-03-05  105.760002  46.439999  114.250000  6753500.0  68136300.0   

Attributes                     High                                Low  \
Symbols             SPY         DIA        QQQ         SPY         DIA   
Date                                                                     
2010-03-01  147709700.0  104.199997  45.490002  112.000000  103.580002   
2010-03-02  160992400.0  104.639999  45.849998  112.739998  103.949997   
2010-03-03  150785000.0  104.769997  45.820000  112.970001  103.830002   
2010-03-04  135770400.0  104.570000  45.810001  112.800003  103.959999   
2010-03-05  176118800.0  105.769997  46.540001  114.339996  104.870003   

Attributes                             future target  
Symbols           QQQ         SPY                     
Date                                                  
2010-03-01  44.950001  111.169998  112.639999      1  
2010-03-02  45.450001  112.000000  114.250000      1  
2010-03-03  45.430000  112.019997  114.269997      1  
2010-03-04  45.400002  112.029999  114.459999      1  
2010-03-05  45.990002  113.099998  114.970001      1

### Training/Test set split

In [0]:
from sklearn import preprocessing
from collections import deque
import random

In [0]:
times = sorted(df.index.values)
last_10pct = times[-int(.1*len(times))]


In [0]:
valid_df = df[(df.index >= last_10pct)]

main_df = df[(df.index < last_10pct)]

In [0]:
def preprocess_df(df):
  df = df.drop('future',1)
  
  for col in df.columns:
    if col != ('target', ''):
      df[col] = df[col].pct_change()
      df.dropna(inplace=True)
      df[col] = preprocessing.scale(df[col].values)
      
  df.dropna(inplace=True)
  
  seq_data = []
  prev_days = deque(maxlen=SEQ_LEN)
  
  for i in df.values:
    prev_days.append([n for n in i[:-1]])
    if len(prev_days) == SEQ_LEN:
      seq_data.append([np.array(prev_days),i[-1]])
  random.shuffle(seq_data)
  
  buys = []
  sells = []
  
  for seq, target in seq_data:
    if target == 0:
      sells.append([seq, target])
    else:
        buys.append([seq, target])
        
  random.shuffle(buys)
  random.shuffle(sells)
  
  lower = min(len(buys),len(sells))
  
  buys = buys[:lower]
  sells = sells[:lower]
  
  seq_data_2 = buys+sells
  random.shuffle(seq_data_2)
  
  X = []
  y = []
  
  for seq, target in seq_data_2:
    X.append(seq)
    y.append(target)
    
  return np.array(X),y
 
 

In [94]:
train_X, train_y = preprocess_df(main_df)
val_X, val_y = preprocess_df(valid_df)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


- to make sure the code does not just guess that to just always buy because there are more buys then sells i made the number of buys equal the number of sells

In [95]:
train_y.count(0)

737

In [96]:
train_y.count(1)

737

## Build the Model
- will use a CuDNNLSTM (if has GPU,use LSTM if no GPU)
- then put through Dense layers
- then output a prediction of if it thiks the price will go up or down

In [0]:
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization

In [0]:
model = Sequential()
model.add(CuDNNLSTM(128, input_shape = (train_X.shape[1:]), return_sequences=True))
model.add(Dropout(.2))
model.add(BatchNormalization())

model.add(CuDNNLSTM(128, input_shape = (train_X.shape[1:]), return_sequences=True))
model.add(Dropout(.2))
model.add(BatchNormalization())


model.add(CuDNNLSTM(128, input_shape = (train_X.shape[1:])))
model.add(Dropout(.2))
model.add(BatchNormalization())

model.add(Dense(50,activation='relu'))
model.add(Dropout(.25))
model.add(BatchNormalization())


model.add(Dense(32,activation='relu'))


model.add(Dense(10,activation='relu'))


model.add(Dense(1, activation=tf.nn.sigmoid))

In [0]:
opt = tf.keras.optimizers.Adam()

In [0]:
model.compile(optimizer=opt,#'adam'
              loss='binary_crossentropy',
              metrics=['acc'])

In [120]:
history = model.fit(
train_X,train_y,batch_size=10,
epochs=10,
validation_data=(val_X,val_y))

Train on 1474 samples, validate on 108 samples
Epoch 1/10
1474/1474 [==============================] - 8s 5ms/sample - loss: 0.7318 - acc: 0.5068 - val_loss: 0.6882 - val_acc: 0.5093
Epoch 2/10
1474/1474 [==============================] - 6s 4ms/sample - loss: 0.7004 - acc: 0.5176 - val_loss: 0.7013 - val_acc: 0.5278
Epoch 3/10
1474/1474 [==============================] - 6s 4ms/sample - loss: 0.7033 - acc: 0.5197 - val_loss: 0.6822 - val_acc: 0.5648
Epoch 4/10
1474/1474 [==============================] - 6s 4ms/sample - loss: 0.6945 - acc: 0.5217 - val_loss: 0.7323 - val_acc: 0.4167
Epoch 5/10
1474/1474 [==============================] - 6s 4ms/sample - loss: 0.7023 - acc: 0.4973 - val_loss: 0.7031 - val_acc: 0.5000
Epoch 6/10
1474/1474 [==============================] - 6s 4ms/sample - loss: 0.6985 - acc: 0.4993 - val_loss: 0.6917 - val_acc: 0.5185
Epoch 7/10
1474/1474 [==============================] - 6s 4ms/sample - loss: 0.6925 - acc: 0.5380 - val_loss: 0.6953 - val_acc: 0.5278
E

### Test Set

In [121]:
start = datetime(2018, 3, 2)

end = datetime(2019, 3, 12)

stocks = ['SPY','DIA','QQQ']

f = web.DataReader(stocks, 'yahoo', start, end)

f.tail()

Attributes        High                                 Low              \
Symbols            DIA         QQQ         SPY         DIA         QQQ   
Date                                                                     
2019-03-06  258.450012  174.750000  279.160004  256.450012  173.279999   
2019-03-07  256.489990  173.179993  276.989990  253.610001  170.789993   
2019-03-08  254.899994  171.259995  274.649994  252.740005  169.339996   
2019-03-11  256.829987  174.850006  278.619995  252.460007  171.850006   
2019-03-12  256.970001  176.139999  280.070007  255.429993  174.770004   

Attributes                    Open                               Close  \
Symbols            SPY         DIA         QQQ         SPY         DIA   
Date                                                                     
2019-03-06  276.970001  258.239990  174.720001  279.149994  256.869995   
2019-03-07  274.070007  256.399994  173.119995  276.829987  254.809998   
2019-03-08  272.420013  252.880005  169.509995  272.940002  254.710007   
2019-03-11  275.230011  252.850006  171.880005  275.260010  256.760010   
2019-03-12  278.850006  256.239990  175.190002  279.059998  255.860001   

Attributes                             Volume                          \
Symbols            QQQ         SPY        DIA         QQQ         SPY   
Date                                                                    
2019-03-06  173.559998  277.329987  3222000.0  27194600.0  75039800.0   
2019-03-07  171.429993  275.010010  4104600.0  39407400.0  94885100.0   
2019-03-08  171.169998  274.459991  5495800.0  39303200.0  85795800.0   
2019-03-11  174.729996  278.440002  5469400.0  30518400.0  65098900.0   
2019-03-12  175.690002  279.489990  6570500.0  33953500.0  79667500.0   

Attributes   Adj Close                          
Symbols            DIA         QQQ         SPY  
Date                                            
2019-03-06  256.326416  173.559998  276.113770  
2019-03-07  254.270767  171.429993  273.803986  
2019-03-08  254.170990  171.169998  273.256378  
2019-03-11  256.216644  174.729996  277.218933  
2019-03-12  255.318542  175.690002  278.264313

In [122]:
df_test = f[['Close',
            'Volume','High','Low']]
df_test['future'] = df_test['Close'][STOCK].shift(-PERIODS_FORWARD)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [123]:
df_test['target'] = list(map(classify,df_test['Close'][STOCK],df_test['future']))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [124]:
test_X, test_y = preprocess_df(df_test)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [0]:
pred =model.predict_classes(test_X)


In [0]:
prob =model.predict(test_X)

In [132]:

from sklearn.metrics import precision_score, recall_score

precision_score(test_y,pred)

0.4819277108433735

In [133]:
recall_score(test_y,pred)

0.4444444444444444

In [134]:
from sklearn.metrics import confusion_matrix

confusion_matrix( test_y,pred)

array([[47, 43],
       [50, 40]])

In [0]:
pred2 = prob > .51

In [138]:
confusion_matrix( test_y,pred2)

array([[78, 12],
       [75, 15]])

In [139]:
precision_score(test_y,pred2)

0.5555555555555556